In [1]:
import json
import csv 
import pandas as pd

from rdflib import Graph, Namespace, URIRef, BNode, Literal
from rdflib.namespace import RDF, FOAF, XSD

In [2]:
data_business = []
csvfile = []
cuisineList = []
existdbList = []
specialList = []

In [3]:
with open('yelp_academic_dataset_business.json', encoding="utf8") as f:
    for line in f:
        data_business.append(json.loads(line))
df = pd.read_csv('csvData.csv')

with open('country_cusine_filtered_list.csv', newline='') as inputfile:
    for row in csv.reader(inputfile):
        cuisineList.append(row[0])
        
with open('category_exist_in_db_List.csv', newline='') as inputfile:
    for row in csv.reader(inputfile):
        existdbList.append(row[0])
        
with open('distinct_with_specialchar.csv', newline='') as inputfile:
    for row in csv.reader(inputfile):
        specialList.append(row[0])

In [4]:
exist1 = pd.read_csv('category_exist_in_db_List.csv',header=None)
exist2 = pd.read_csv('category_exist_in_db_List_v2.csv', header=None)
exist3 = pd.read_csv('category_exist_in_db_List_v3.csv', header=None)

frames = [exist1, exist2, exist3]
result = pd.concat(frames)
result = result.reset_index(drop=True)
#exist_v = exist2.add(exist3, fill_value=0)

display(result)

,0,1
0,Doctors,NaN
1,Acupuncture,NaN
2,Nutritionists,NaN
3,Notaries,NaN
4,Shopping,NaN
...,...,...
770,Gerontologists,Gerontologist
771,Otologists,Otologist
772,Parklets,Parklet
773,Neurotologists,Neurotologist


In [5]:
def string_seperate(string):
    n = 0
    cate_list = []
    a_list = string.split(", ")
    for element in a_list:
        cate_list.append(element)
    n = n +1
    return cate_list

def code_to_state(code):
    n = 0
    for index, row in df.iterrows():
        if str(code) == str(df.loc[n,'Code']):
            return str(df.loc[n,'State'])
        #str(df.loc[n,])
        n = n + 1
        

def change_format_to_db(catagory):
    #this function takes a string in catagory from yelp
    #and gives the correct format to put in dbpedia
    
    #Everthing in string to lowercase
    newcatagory = catagory.lower()
    
    #capitalize first letter 
    newcatagory = catagory.capitalize()
    
    #replace empty with _
    newcatagory = catagory.replace(" ", "_")
    
    return newcatagory

In [6]:
schema = Namespace("https://schema.org/")
yelp_business = Namespace("https://www.yelp.com/biz/")
db_state = Namespace("http://dbpedia.org/resource/")
g_maps = Namespace("")

g = Graph()

In [7]:
def triple_maker_name():
    #for each name that exist for a business, add a triple to the graph
    n = 0
    for i in data_business:
        if len(data_business[n]['name']) > 0 and data_business[n]['name'] != None:
            #append to graph
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.legalName, Literal(data_business[n]['name'])))
        n = n + 1
    print('name done')

In [8]:
def triple_maker_address():
    #for each address that exist for a business, add a triple to the graph
    n = 0
    for i in data_business:
        #append to graph
        if len(data_business[n]['address']) > 0:
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.streetAddress, Literal(data_business[n]['address'])))
        n = n + 1
    print('address done')

In [9]:
def triple_maker_state():
    n = 0
    for i in data_business:
        #append to graph
        if len(data_business[n]['state']) > 0:
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.addressRegion, URIRef(db_state + str(code_to_state(data_business[n]['state'])))))
        n = n + 1
        
    print('state done')

In [10]:
def triple_maker_postal():
    n = 0
    for i in data_business:
        #append to graph
        if len(data_business[n]['postal_code']) > 0:
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.postalCode, Literal(data_business[n]['postal_code'])))
        n = n + 1
        
    print('postal done')

In [11]:
def triple_maker_latlong():
    n = 0
    for i in data_business:
        #append to graph
        if data_business[n]['latitude'] != None and len(str(data_business[n]['longitude'])) != None :
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.geo, URIRef(g_maps + str(data_business[n]['latitude']))))
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.geo, URIRef(g_maps + str(data_business[n]['longitude']))))
        n = n + 1
    print('lat+long done')

In [12]:
def triple_maker_stars():
    n = 0
    for i in data_business:
        #append to graph
        if len(str(data_business[n]['stars'])) > 0:
            g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.starRating, Literal(data_business[n]['stars'])))
        n = n + 1
    print('stars done')

In [13]:
def triple_maker_cg_cuisine():
    n = 0
    for i in data_business:
        if data_business[n]['categories'] != None:
            #split the string in categories to a list, create list as catagory_list
            catagory_list = string_seperate(data_business[n]['categories'])
                
            #for every item in catagory_list, compare it with the cuisineList
            for item in range(len(cuisineList)):
                for element in range(len(catagory_list)):
                    #the item in category exist, add it to the graph
                    if catagory_list[element] == cuisineList[item]:
                        #cause the cuisines exist as country+_cuisine in dbpedia
                        #so every country get added a _cuisine after
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.category, URIRef(db_state + str(cuisineList[element])+ str('_cuisine'))))
        n = n + 1
    print('category for cuisine done')  

In [14]:
def triple_maker_cg_in_db_exist():
                  
    n = n + 1
    for i in data_business:
        if data_business[n]['categories'] != None:
            #split the string in categories to a list, create list as catagory_list
            catagory_list = string_seperate(data_business[n]['categories'])
                
            #for every item in catagory_list, compare it with a elemenent that exist in dbpedia
            item1 = 0
            for item in range(len(catagory_list)):
                element1 = 0
                for element in range(len(existdbList)):
                    #the item in category exist, add it to the graph
                    if str(catagory_list[item]) == str(existdbList[element]):
                        #the format in dbpedia is first letter is upper case
                        #and the rest of the letters are lowercase
                        #so every string goes through the function change_format_to_db
                        #in order to be the correct format
                        
                        g.add((URIRef(yelp_business + data_business[n]['business_id']), schema.category, URIRef(db_state + change_format_to_db(existdbList[element]))))
    print('category that exist in db done')

In [15]:
g2 = Graph()

def triple_maker_cg_in_db_exist_2():
    category_exist = 0    
    category_not_exist = 0  
    
    for n in range(len(data_business)):
        if data_business[n]['categories'] != None:
            #split the string in categories to a list, create list as catagory_list
            catagory_list = string_seperate(data_business[n]['categories'])
            #for every item in catagory_list, compare it with the dbpedia
            item1 = 0
            for item in range(len(catagory_list)):
                element1 = 0
                for index, row in df.iterrows():
                    #the item in category exist, add it to the graph
                    if str(catagory_list[item]) == str(row[0]):
                        
                        #the format in dbpedia is first letter is upper case
                        #and the rest of the letters are lowercase
                        #so every string goes through the function change_format_to_db
                        #in order to be the correct format
                        #g2.add((URIRef(yelp_business + data_business[n]['business_id']), schema.category, URIRef(db_state + change_format_to_db(existdbList[element]))))
                        category_exist = category_exist + 1
                    else:
                        category_not_exist = category_not_exist + 1
    print('category_2 that exist in db done')

In [26]:
def triple_maker_att_RestaurantsTakeOut():
    for n in range(0,10):
        if data_business[n]['attributes'] != None:
            print(data_business[n]['attributes'])
        
            if data_business[n]['attributes'].__contains__('RestaurantsTakeOut'):
                print(data_business[n]['attributes']['RestaurantsTakeOut'])
            else:
                print("Key does not exist")
        n = n +1

In [27]:
triple_maker_att_RestaurantsTakeOut()

{'ByAppointmentOnly': 'True'}
Key does not exist
{'BusinessAcceptsCreditCards': 'True'}
Key does not exist
{'BikeParking': 'True', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsPriceRange2': '2', 'CoatCheck': 'False', 'RestaurantsTakeOut': 'False', 'RestaurantsDelivery': 'False', 'Caters': 'False', 'WiFi': "u'no'", 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'WheelchairAccessible': 'True', 'HappyHour': 'False', 'OutdoorSeating': 'False', 'HasTV': 'False', 'RestaurantsReservations': 'False', 'DogsAllowed': 'False', 'ByAppointmentOnly': 'False'}
False
{'RestaurantsDelivery': 'False', 'OutdoorSeating': 'False', 'BusinessAcceptsCreditCards': 'False', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'BikeParking': 'True', 'RestaurantsPriceRange2': '1', 'RestaurantsTakeOut': 'True', 'ByAppointmentOnly': 'False', 'WiFi': "u'free'", 'Alcohol': "u'none'", 'Caters': 'True'}


In [17]:
lol = 10

print(type(data_business[lol]['attributes']))
print(data_business[lol]['attributes'])
print(type(string_seperate(data_business[lol]['attributes']['BusinessParking'])))
print(string_seperate(data_business[lol]['attributes']['BusinessParking']))

<class 'dict'>
{'RestaurantsPriceRange2': '2', 'BikeParking': 'True', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}"}
<class 'list'>
["{'garage': False", "'street': False", "'validated': False", "'lot': False", "'valet': False}"]


In [18]:
'''
triple_maker_name()
triple_maker_address()
triple_maker_state()
triple_maker_postal()
triple_maker_latlong()
triple_maker_stars()
triple_maker_cg_cuisine()
triple_maker_cg_in_db_exist()
'''

'\ntriple_maker_name()\ntriple_maker_address()\ntriple_maker_state()\ntriple_maker_postal()\ntriple_maker_latlong()\ntriple_maker_stars()\ntriple_maker_cg_cuisine()\ntriple_maker_cg_in_db_exist()\n'

In [19]:
#g.serialize(destination="Business_v2.ttl")

In [20]:
#triple_maker_cg_in_db_exist_2()

#print(category_exist)
#print(category_not_exist)